In [ ]:
import pandas as pd
import funzioni as fx
from tensorflow.keras.models import load_model
from tensorflow_addons.metrics import F1Score
import yfinance as yf
from multiprocessing import Pool, cpu_count, Manager, Value
import tensorflow as tf

# Ottiene la lista di tutte le GPU fisiche disponibili
gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
    try:
        # Imposta TensorFlow per utilizzare tutte le GPU visibili
        tf.config.experimental.set_visible_devices(gpus, 'GPU')
        
        for gpu in gpus:
            # Imposta TensorFlow per allocare dinamicamente la memoria su ogni GPU disponibile
            tf.config.experimental.set_memory_growth(gpu, True)
        
    except RuntimeError as e:
        # L'eccezione viene sollevata se si tenta di impostare la visibilità delle GPU o la crescita della memoria
        # dopo che la sessione di TensorFlow è stata inizializzata
        print(e)

def scarica(nome_simbolo):
    try:
        ticker = yf.download(nome_simbolo, start='2021-01-01', end='2023-12-31', progress=False)
        if ticker["Close"].iloc[-1] >= 1:
            ticker.index = ticker.index.date
            ticker = fx.crea_indicatori(ticker)
            ticker = fx.imposta_target(ticker)
            ticker.dropna(axis=0, inplace=True)
            _, X, Y, _ = fx.to_XY(ticker, fx.features_prezzo, fx.features_da_scalare_singolarmente, fx.features_meno_piu, fx.features_candele, fx.features_no_scala, fx.elenco_targets, fx.n_timesteps, fx.giorni_previsione, 0)
            pred = model.predict(X)
            screener.append([nome_simbolo, pred[-1, 0]])  
        return nome_simbolo, X, Y, ""
    except Exception as e:
        return nome_simbolo, None, None, str(e)

def callback_result(result, totale_processati, tot_tickers):
    nome_simbolo, X, Y, err = result
    if X is not None and Y is not None:
        print(f'X.shape:{X.shape}')
        print(f'Y.shape:{Y.shape}')
        print("\033[42m" + f"{totale_processati.value}/{tot_tickers}) Completato ticker {nome_simbolo}" + "\033[0m")
    else:
        print(err)

    with totale_processati.get_lock(): 
        totale_processati.value += 1

f1_score = F1Score(num_classes=1, average='macro', threshold=0.5)
model = load_model("model.h5", custom_objects={'Addons>F1Score': f1_score})

lista_tickers = pd.read_parquet("lista_ticker.parquet")
tot_tickers = len(lista_tickers)

manager = Manager()
screener = manager.list()

totale_processati = Value('i', 0)  

with Pool(cpu_count()) as p:
    for i in range(0, tot_tickers):
        nome_simbolo = lista_tickers.iloc[i]["Ticker"]
        task = (nome_simbolo,)
        p.apply_async(scarica, args=task, callback=lambda result: callback_result(result, totale_processati, tot_tickers))

    p.close()
    p.join()

print("Salvataggio file")
screener_df = pd.DataFrame(list(screener), columns=["Ticker", "Prediction"])
screener_df.to_excel("screener.xlsx")

print("fine")

2023-11-08 09:18:52.155101: F tensorflow/stream_executor/cuda/cuda_driver.cc:152] Failed setting context: CUDA_ERROR_NOT_INITIALIZED: initialization error
2023-11-08 09:18:52.460792: F tensorflow/stream_executor/cuda/cuda_driver.cc:152] Failed setting context: CUDA_ERROR_NOT_INITIALIZED: initialization error
2023-11-08 09:18:52.482244: F tensorflow/stream_executor/cuda/cuda_driver.cc:152] Failed setting context: CUDA_ERROR_NOT_INITIALIZED: initialization error
2023-11-08 09:18:52.500457: F tensorflow/stream_executor/cuda/cuda_driver.cc:152] Failed setting context: CUDA_ERROR_NOT_INITIALIZED: initialization error
2023-11-08 09:18:52.519863: F tensorflow/stream_executor/cuda/cuda_driver.cc:152] Failed setting context: CUDA_ERROR_NOT_INITIALIZED: initialization error
2023-11-08 09:18:52.623086: F tensorflow/stream_executor/cuda/cuda_driver.cc:152] Failed setting context: CUDA_ERROR_NOT_INITIALIZED: initialization error
2023-11-08 09:18:52.623443: F tensorflow/stream_executor/cuda/cuda_dri

single positional indexer is out-of-bounds


2023-11-08 09:19:04.488110: F tensorflow/stream_executor/cuda/cuda_driver.cc:152] Failed setting context: CUDA_ERROR_NOT_INITIALIZED: initialization error
2023-11-08 09:19:04.661388: F tensorflow/stream_executor/cuda/cuda_driver.cc:152] Failed setting context: CUDA_ERROR_NOT_INITIALIZED: initialization error
2023-11-08 09:19:04.767935: F tensorflow/stream_executor/cuda/cuda_driver.cc:152] Failed setting context: CUDA_ERROR_NOT_INITIALIZED: initialization error
2023-11-08 09:19:04.863531: F tensorflow/stream_executor/cuda/cuda_driver.cc:152] Failed setting context: CUDA_ERROR_NOT_INITIALIZED: initialization error
2023-11-08 09:19:04.872866: F tensorflow/stream_executor/cuda/cuda_driver.cc:152] Failed setting context: CUDA_ERROR_NOT_INITIALIZED: initialization error
2023-11-08 09:19:05.153811: F tensorflow/stream_executor/cuda/cuda_driver.cc:152] Failed setting context: CUDA_ERROR_NOT_INITIALIZED: initialization error
2023-11-08 09:19:05.585528: F tensorflow/stream_executor/cuda/cuda_dri

Process ForkPoolWorker-109:
KeyboardInterrupt
Traceback (most recent call last):
  File "/usr/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/usr/lib/python3.9/multiprocessing/queues.py", line 364, in get
    with self._rlock:
  File "/usr/lib/python3.9/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()


KeyboardInterrupt: 